In [1]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"]='0'
import tensorflow
import cv2
import numpy as np

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model

from pathlib import Path
from PIL import Image
from itertools import product

# To import of OpenSlide.
OPENSLIDE_PATH = r'C:\Users\ga-steynlab-03\openslide-bin-4.0.0.3-windows-x64\bin'
#import os
if hasattr(os, 'add_dll_directory'):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [2]:
################################################################################
# script to Load .ndpi, split it into tiles then resize (0.5 micron) each tile #
################################################################################

def split_and_resize_tiles(ndpi_file, output_dir, tile_size=256, target_resolution=0.5):
    # Open the NDPI file
    slide = openslide.open_slide(ndpi_file)
    
    # Calculate the number of levels required to achieve the target resolution
    original_resolution = float(slide.properties.get("openslide.mpp-x"))
    resize_factor = original_resolution / target_resolution
    level = slide.get_best_level_for_downsample(resize_factor)
    downsample_factor = slide.level_downsamples[level]

     # Create output folder for the current NDPI file
    file_name = os.path.splitext(os.path.basename(ndpi_file))[0]
    #file_output_dir = os.path.join(output_dir, file_name)
    #os.makedirs(file_output_dir, exist_ok=True)
    
    # Iterate through each tile  ## (0, h-h%d, d)
    for y in range(0, slide.level_dimensions[level][1], tile_size): 
        for x in range(0, slide.level_dimensions[level][0], tile_size):
            # Read tile from slide
            tile = slide.read_region((x, y), level, (tile_size, tile_size))
            
            # Resize tile
            new_size = (int(tile_size / downsample_factor), int(tile_size / downsample_factor))
            resized_tile = tile.resize(new_size, resample=Image.BICUBIC)
            
            # Save resized tile
            tile_name = file_name + '_' + str(x).zfill(3) + '_' + str(y).zfill(3) + '.png'
            #tile_name = f"tile_{x}_{y}.png"
            tile_path = os.path.join(output_dir, tile_name)
            resized_tile.save(tile_path)
    
    # Close the slide
    slide.close()

# Example usage
#ndpi_file = "C:/Users/ga-steynlab-03/Demo_TB/wsi_ndpi/SL27-1.ndpi"
#output_dir = "C:/Users/ga-steynlab-03/Demo_TB/wsi_ndpi_tiles"
#os.makedirs(output_dir, exist_ok=True)

#split_and_resize_tiles(ndpi_file, output_dir)
#print("Tiles resized and saved in:", tile_path)

In [ ]:
if __name__ == '__main__':
    
    #### Point to the folder containing wsi
    ### use if the WSI are located on the same directory as the script (working directory).
    #folder_name = 'wsi_ndpi'
    #folder_path = os.path.join(os.getcwd(), folder_name)

    ### Specify the Path if WSIs are stored elsewhere (e.g., external disk or file server)
    folder_path = 'X:/KAPONGO/0 MSc/WSI for Testing/WSI - ndpi'
    
    # Output folder...
    output_folder = 'ndpi_tiles-v2'
    output_path = os.path.join(os.getcwd(), output_folder)
    #output_path.mkdir(exist_ok=True)
    
    # loop through the ndpi in the folder...
    for ndpi_file in Path(folder_path).glob('*.ndpi'):
        # Create output folder for each wsi
        output_tile = Path(output_path).joinpath(ndpi_file.stem)
        output_tile.mkdir(exist_ok=True)
        
        split_and_resize_tiles(ndpi_file, output_tile)

    ################